In [8]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import tensorflow as tf
import pandas as pd
import pickle
import random
import json


In [9]:
intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [10]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype = object)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [12]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [13]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)

Epoch 1/100
21/21 [==============================] - 0s 974us/step - loss: 3.1055 - accuracy: 0.0980
Epoch 2/100
21/21 [==============================] - 0s 1ms/step - loss: 3.0546 - accuracy: 0.0882
Epoch 3/100
21/21 [==============================] - 0s 1ms/step - loss: 3.0443 - accuracy: 0.1078
Epoch 4/100
21/21 [==============================] - 0s 1ms/step - loss: 2.9258 - accuracy: 0.1078
Epoch 5/100
21/21 [==============================] - 0s 1ms/step - loss: 2.8355 - accuracy: 0.1863
Epoch 6/100
21/21 [==============================] - 0s 1ms/step - loss: 2.7141 - accuracy: 0.2353
Epoch 7/100
21/21 [==============================] - 0s 1ms/step - loss: 2.6187 - accuracy: 0.2157
Epoch 8/100
21/21 [==============================] - 0s 1ms/step - loss: 2.4402 - accuracy: 0.3627
Epoch 9/100
21/21 [==============================] - 0s 1ms/step - loss: 2.4230 - accuracy: 0.3333
Epoch 10/100
21/21 [==============================] - 0s 1ms/step - loss: 2.2670 - accuracy: 0.3137
Epoch 1

21/21 [==============================] - 0s 1ms/step - loss: 0.2529 - accuracy: 0.9314
Epoch 83/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0877 - accuracy: 0.9902
Epoch 84/100
21/21 [==============================] - 0s 1ms/step - loss: 0.1624 - accuracy: 0.9510
Epoch 85/100
21/21 [==============================] - 0s 1ms/step - loss: 0.1453 - accuracy: 0.9608
Epoch 86/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0842 - accuracy: 0.9804
Epoch 87/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0987 - accuracy: 0.9706
Epoch 88/100
21/21 [==============================] - 0s 1ms/step - loss: 0.1685 - accuracy: 0.9412
Epoch 89/100
21/21 [==============================] - 0s 1ms/step - loss: 0.1034 - accuracy: 0.9902
Epoch 90/100
21/21 [==============================] - 0s 1ms/step - loss: 0.1734 - accuracy: 0.9608
Epoch 91/100
21/21 [==============================] - 0s 1ms/step - loss: 0.1629 - accuracy: 0.9608
Epoch 92/100


In [14]:

model.save('chatbot.h5',model)
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "learning_chatbot.pkl", "wb" ) )